In [1]:
import argparse
import logging
import subprocess

from agent.prompt import PROMPT
from agent.utils import process_repository


In [34]:
class SimpleAgentV2:
    def __init__(self):
        self.api_key = self.get_api_key()
        self.llm = llm = ChatOpenAI(temperature=0, model="gpt-4-turbo")
        self.conv = ConversationChain(
            llm=self.llm,
            memory=ConversationBufferMemory()
        )
    
    @staticmethod
    def get_api_key():
        load_dotenv()
        return os.getenv("OPENAI_API_KEY")
    
    def run(self, prompt):
        response = self.conv(prompt)
        return response

task_description = """
I am running into an issue where the time my react page is 
supposed to display is just 0 instead of the current timestamp. Fix this issue.
"""

task_name = "fix_react_timestamp"
use_mock = False

repository_context = process_repository("/Users/inafi/Develop/Github/dave/apps/dave/example-task")

agent_prompt = PROMPT.format(task_description=task_description, repository_context=repository_context)

agent = SimpleAgentV2()


In [35]:
response = agent.run(agent_prompt)

In [45]:
agent.run("Which files paths should you modify?")

{'input': 'Which files paths should you modify?',
 'history': 'Human: \n================================Start=============================\nYou are a software engineer tasked with resolving GitHub issues and automating tasks on a system. You have access to a MacOS terminal, where an assistant executes commands, summarizes results, and provides the output back to you. You can utilize all standard tools commonly available on such systems without needing special access tokens. Your tasks can include file creation, script execution, or using network utilities like curl to interact with APIs.\n\nConsider replicating browser behaviors in terminal commands, such as handling redirects in curl. \n\nTask Description:\nThe GitHub issue or task description is \nI am running into an issue where the time my react page is \nsupposed to display is just 0 instead of the current timestamp. Fix this issue.\n.\n\n\nYou have full access to the host machine, allowing you to create or download files, initiat